# XGB with data cleaning and feature engineering.

In [1]:
# sub.to_csv('xgb_starter.csv.gz', index=False, float_format='%.4g', compression='gzip')

## Intro:

In this notebook, I will try to a) import and clean data and b) run an XGB decision tree regression model to make Kaggle predictions.

Shoutout to these helpful scripts:

- https://www.kaggle.com/infinitewing/xgboost-without-outliers-lb-0-06463

In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from datetime import datetime
import re

In [3]:
np.random.seed(1234)

In [4]:
all_features = pd.read_csv('Data/binarized_numerics.csv')
train = pd.read_pickle('Data/train')
samp = pd.read_pickle('Data/sample_submission')

In [5]:
all_features.shape

(2985217, 119)

In [6]:
train.shape

(90275, 3)

In [7]:
samp.shape

(2985217, 7)

In [8]:
samp.rename(columns={'ParcelId': 'parcelid'}, inplace=True)
samp.head()

,parcelid,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


In [9]:
train_df = train.merge(all_features, how='left', on='parcelid')

I encountered a small problem here...there are multiple sales on a few properties, so the parcelid shows up multiple times. This is fine, but we need to get rid of the duplicate parcelid when we make our final predictions.

In [10]:
predict_df = samp.merge(train_df, on='parcelid', how='left')

In [11]:
train.parcelid.value_counts().head()

11842707    3
12613442    2
12032773    2
11729067    2
11845988    2
Name: parcelid, dtype: int64

In [12]:
# Check on structure
if predict_df.shape[0] - sum(predict_df['logerror'].isnull()) == train_df.shape[0]:
    print('All good! Training set contains all of the rows with non-null values of logerror in the predict set.')
else:
    print('Data dimensions are off...check and resolve this problem.')

All good! Training set contains all of the rows with non-null values of logerror in the predict set.


## (Potential to do?) Add date features

In [13]:
# train_df['transactiondate'] = pd.to_datetime(train_df['transactiondate'])
# train_df['year'] = train_df['transactiondate'].dt.year
# train_df['month'] = train_df['transactiondate'].dt.month
# train_df['day'] = train_df['transactiondate'].dt.day

## XGB model fit

In [14]:
train_df = train_df[ train_df.logerror > -0.4 ]
train_df = train_df[ train_df.logerror < 0.4 ]

In [15]:
# copy train_df
x_train_full = train_df

In [16]:
# pop off labels to predict
labels = x_train_full.pop('logerror') 
labels[0:4]

0    0.0276
1   -0.1684
2   -0.0040
3    0.0218
Name: logerror, dtype: float64

In [17]:
# rename standard names
x_train, y_train  = x_train_full, labels

In [18]:
# set y_mean as the base prediction
y_mean = labels.mean()

In [19]:
x_train.columns

Index(['parcelid', 'transactiondate', 'Unnamed: 0', 'airconditioningtypeid_0',
       'airconditioningtypeid_1', 'airconditioningtypeid_2',
       'architecturalstyletypeid_0', 'architecturalstyletypeid_1',
       'architecturalstyletypeid_2', 'architecturalstyletypeid_3',
       ...
       'N-LivingAreaProp', 'N-ValueRatio', 'N-ValueProp', 'N-location',
       'N-location-2', 'N-ExtraSpace', 'N-zip_count', 'N-TaxScore',
       'N-Avg-structuretaxvaluedollarcnt', 'N-structuretaxvaluedollarcnt-2'],
      dtype='object', length=120)

In [20]:
# drop the non-predicted columns from our training set
x_train = train_df.drop(['parcelid', 'Unnamed: 0','transactiondate'], axis=1)

In [21]:
# keep these columns to train our models on
cols_to_train_on = x_train.columns
x_test = predict_df[cols_to_train_on]

In [22]:
# convert na to -1 for train and test sets
for c in x_train.columns:
    x_train[c]=x_train[c].fillna(-1)
    
for c in x_test.columns:
    x_test[c]=x_test[c].fillna(-1)

/Users/wnowak/anaconda/envs/py35/lib/python3.5/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [23]:
# xgboost params
xgb_params = {
    'eta': 0.06,
    'max_depth': 10,
    'subsample': 0.75,
    'objective': 'reg:linear',
    'eval_metric': 'mae',
    'base_score': y_mean,
    'silent': 1
}

In [24]:
# create xgb Dmatrices
d_train = xgb.DMatrix(x_train, y_train)
d_test = xgb.DMatrix(x_test)

In [25]:


# cross-validation
cv_result = xgb.cv(xgb_params, 
                   d_train, 
                   nfold=5,
                   num_boost_round=200,
                   early_stopping_rounds=15,
                   verbose_eval=5, 
                   show_stdv=False
                  )
num_boost_rounds = len(cv_result)
print(num_boost_rounds)

[0]	train-mae:0.0527166	test-mae:0.05283
[5]	train-mae:0.0518766	test-mae:0.0525632
[10]	train-mae:0.0512272	test-mae:0.052438
[15]	train-mae:0.0506592	test-mae:0.0523882
[20]	train-mae:0.0501996	test-mae:0.052379
[25]	train-mae:0.0497948	test-mae:0.0523662
[30]	train-mae:0.0494024	test-mae:0.0523716
[35]	train-mae:0.0490604	test-mae:0.0523654
[40]	train-mae:0.0487308	test-mae:0.0523666
[45]	train-mae:0.0484324	test-mae:0.0523794
[50]	train-mae:0.0481506	test-mae:0.0523966
39


In [26]:
# train model
model = xgb.train(dict(xgb_params, silent=1), 
                  d_train, 
#                   evals = watchlist,
                  num_boost_round=num_boost_rounds)

In [27]:
pred = model.predict(d_test)

In [28]:
y_pred = []
for i,predict in enumerate(pred):
    y_pred.append(str(round(predict,4)))

In [29]:
y_pred=np.array(y_pred)

In [30]:
output = pd.DataFrame({'ParcelId': predict_df['parcelid'].astype(np.int32),
        '201610': y_pred, '201611': y_pred, '201612': y_pred,
        '201710': y_pred, '201711': y_pred, '201712': y_pred})

In [31]:
output[output.ParcelId == 11842707]
# .ParcelId.value_counts()

,201610,201611,201612,201710,201711,201712,ParcelId
1194796,0.028,0.028,0.028,0.028,0.028,0.028,11842707
1194797,0.028,0.028,0.028,0.028,0.028,0.028,11842707
1194798,0.028,0.028,0.028,0.028,0.028,0.028,11842707


We have duplicate entries, as you can see above:

In [32]:
submission = output.drop_duplicates()

In [33]:
output.head()

,201610,201611,201612,201710,201711,201712,ParcelId
0,0.0107,0.0107,0.0107,0.0107,0.0107,0.0107,10754147
1,0.0107,0.0107,0.0107,0.0107,0.0107,0.0107,10759547
2,0.0107,0.0107,0.0107,0.0107,0.0107,0.0107,10843547
3,0.0107,0.0107,0.0107,0.0107,0.0107,0.0107,10859147
4,0.0107,0.0107,0.0107,0.0107,0.0107,0.0107,10879947


In [34]:
cols = submission.columns.tolist()
cols = cols[-1:] + cols[:-1]
output = submission[cols]
from datetime import datetime
submission.to_csv('sub{}.csv'.format(datetime.now().strftime('%Y%m%d_%H%M%S')), index=False)